In [2]:
import numpy as np
import sounddevice as sd
from scipy.signal import hilbert

In [ ]:
# Parameters for FSK
SAMPLE_RATE = 44100  # Standard audio sampling rate
DURATION = 0.02  # Duration of each bit in seconds
FREQ_0 = 2000  # Frequency for bit 0
FREQ_1 = 4000  # Frequency for bit 1

# Convert text to binary
def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text)

# Generate FSK-modulated waveform
def generate_fsk_wave(binary_data):
    t = np.linspace(0, DURATION, int(SAMPLE_RATE * DURATION), False)
    waveform = np.concatenate([
        0.5 * np.sin(2 * np.pi * (FREQ_1 if bit == '1' else FREQ_0) * t)
        for bit in binary_data
    ])
    return waveform

# Play the sound and return the waveform
def text_to_fsk_sound(text):
    binary_data = text_to_binary(text)  # Convert text to binary
    print(f"Text: {text}")
    print(f"Binary: {binary_data}")

    waveform = generate_fsk_wave(binary_data)  # Generate sound wave
    print("Playing FSK-modulated sound...")
    
    sd.play(waveform, samplerate=SAMPLE_RATE)  # Play the sound
    sd.wait()
    
    return waveform  # Return the waveform

# Decode FSK waveform back to binary
def fsk_wave_to_binary(waveform):
    samples_per_bit = int(SAMPLE_RATE * DURATION)  # Number of samples per bit
    binary_data = ""

    for i in range(0, len(waveform), samples_per_bit):
        segment = waveform[i:i+samples_per_bit]  # Extract bit-sized segment

        # Hilbert transform to extract envelope (frequency detection)
        analytic_signal = hilbert(segment)
        amplitude_envelope = np.abs(analytic_signal)
        
        # Determine dominant frequency using zero-crossing rate
        zero_crossings = np.where(np.diff(np.sign(segment)))[0]
        frequency = len(zero_crossings) / (2 * DURATION)  # Approximate frequency

        # Assign bit based on closest frequency
        if abs(frequency - FREQ_1) < abs(frequency - FREQ_0):
            binary_data += "1"
        else:
            binary_data += "0"

    return binary_data

# Convert binary back to text
def binary_to_text(binary_data):
    chars = [binary_data[i:i+8] for i in range(0, len(binary_data), 8)]
    return ''.join(chr(int(char, 2)) for char in chars if len(char) == 8)

# Decode FSK waveform back to text
def decode_fsk_sound(waveform):
    binary_data = fsk_wave_to_binary(waveform)
    text = binary_to_text(binary_data)
    return text


if __name__ == "__main__":
    
    wave = text_to_fsk_sound("hello")  # Converts "hello" to FSK sound and returns the waveform
    decoded_text = decode_fsk_sound(wave)  # Decode the waveform back to text
    print(f"Decoded Text: {decoded_text}")

Text: hello
Binary: 0110100001100101011011000110110001101111
Playing FSK-modulated sound...
Decoded Text: hello
